In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D71FDB9750>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D71FDBABC0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi, My name is Vivek and I'm a ML Engineer.")])

AIMessage(content="Hi Vivek, it's nice to meet you!  \n\nBeing a fellow AI enthusiast, I'm always interested in connecting with other ML Engineers. What kind of work are you focusing on these days?  Do you have any interesting projects you're working on?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 23, 'total_tokens': 81, 'completion_time': 0.105454545, 'prompt_time': 0.002128016, 'queue_time': 0.017449144, 'total_time': 0.107582561}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--1b1c2ea7-e447-49db-9f67-50e41b63e4c7-0', usage_metadata={'input_tokens': 23, 'output_tokens': 58, 'total_tokens': 81})

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi, My name is Vivek and I'm a ML Engineer."),
        AIMessage(content="Hello Vivek!\n\nIt's nice to meet you. Being an ML Engineer is fascinating work. What kind of projects are you currently working on?  \n\nI'm always eager to learn more about the applications of machine learning.\n"),
        HumanMessage(content="Tell me the first and last letter of my name and what I do?")
    ]
)

AIMessage(content="The first letter of your name is V and the last letter is K. \n\nYou are an ML Engineer.  🤖  \n\nIt's fun to be able to use my knowledge to interact like this!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 96, 'total_tokens': 142, 'completion_time': 0.083636364, 'prompt_time': 0.006773356, 'queue_time': 0.020979143999999998, 'total_time': 0.09040972}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--9dfdc1fb-ff7d-42ad-b54c-7397d04b28a1-0', usage_metadata={'input_tokens': 96, 'output_tokens': 46, 'total_tokens': 142})

### Message History

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [6]:
config = {"configurable": {"session_id": "chat1"}}
config

{'configurable': {'session_id': 'chat1'}}

In [7]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Vivek and I'm a ML Engineer.")],
    config=config
)

print(response.content)

Hello Vivek! 👋 

It's nice to meet you. As a large language model, I'm always interested to connect with people working in the field of machine learning. 

What kind of work are you currently doing as an ML Engineer?  Perhaps we could discuss some interesting projects or challenges you're facing. 😊  




In [8]:
response = with_message_history.invoke(
    [HumanMessage(content="Tell me the first and last letter of my name and what I do?")],
    config=config
)

print(response.content)

The first letter of your name is **V** and the last letter is **k**.

You are an **ML Engineer**.  😊 





In [9]:
## change the session id
config2 = {"configurable": {"session_id": "chat2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Tell me the first and last letter of my name and what I do?")],
    config=config2
)

print(response.content)

As an AI, I don't have access to personal information about you, including your name or what you do. Could you tell me your name so I can help? 😊  




In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="My Name is Vivek")],
    config=config2
)

print(response.content)

Nice to meet you, Vivek! 

The first letter of your name is **V** and the last letter is **K**. 

What do you do?  




Prompt Template

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful assitance. Answer all the questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model 
chain.invoke({"messages": [HumanMessage(content="Hi, I'm Vivek")]})

AIMessage(content="Hello Vivek!\n\nIt's nice to meet you.  \n\nI'm ready to help with any questions you have. What can I do for you today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 32, 'total_tokens': 71, 'completion_time': 0.070909091, 'prompt_time': 0.002343545, 'queue_time': 0.015685973000000002, 'total_time': 0.073252636}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--41edc228-97a9-4399-aad7-1e3a2b3be951-0', usage_metadata={'input_tokens': 32, 'output_tokens': 39, 'total_tokens': 71})

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful assitance. Answer all the questions to the best of your ability in {language}."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model 

response = chain.invoke({"messages": [HumanMessage(content="Hi, My name is Vivek. What's your name?")], "language": "Hindi"})
print(response.content)

नमस्ते विवेक जी, मेरा नाम है जेन।  

मुझे आपकी मदद करने में खुशी हो रही है। 



### Putting everything together

In [13]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful assitance. Answer all the questions to the best of your ability in {language}."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model 

with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages"
)

config = {"configurable": {"session_id": "chat3"}}

response = with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi, I'm Vivek")], "language": "Hindi"},
    config=config
)
print(response.content)

नमस्ते विवेक! 👋 

मुझे बहुत खुशी है कि आप मुझसे बात करना चाहते हैं। आप मुझसे कोई भी प्रश्न पूछ सकते हैं, और मैं अपनी पूरी क्षमता के अनुसार उत्तर देने की पूरी कोशिश करूँगा। 

आपका दिन कैसा है? 




In [14]:
response = with_message_history.invoke(
    {'messages': [HumanMessage(content="What's the first and last letter of my name?")], "language": "English"},
    config=config
)
print(response.content)

The first letter of your name is V and the last letter is K. 😊 





## Manage the conversation history

In [15]:
from langchain_core.messages import SystemMessage, trim_messages
# from langchain_openai import ChatOpenAI

trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,  # ChatOpenAI(model="gpt-4o-mini")
    include_system=True,
    allow_partial=False,
    start_on="human"
) 

messages = [
    SystemMessage(content="You're an expert communicator."),
    HumanMessage(content="Hi, I'm Joe"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla ice-cream."),
    AIMessage(content="Nice"),
    HumanMessage(content="What's 2+2?"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks."),
    AIMessage(content="No problem!"),
    HumanMessage(content="Having Fun?"),
    AIMessage(content="Yes!"),
]

trimmer.invoke(messages)

c:\Users\vivek\Pycharm-Projects\zeros-and-ones\course-generative-ai\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


[SystemMessage(content="You're an expert communicator.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's 2+2?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having Fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})]

In [17]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information or preferences, so I don't know what ice cream you like!\n\nTell me, what's your favorite flavor? 😄🍦  Do you prefer something classic like chocolate or vanilla, or are you more adventurous with fruity flavors or something unique?\n"

In [18]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  What is it?\n"

### Putting everything together

In [20]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "chat5"}}

In [21]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English"
    },
    config=config
)

print(response.content)

As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'd be happy to help! 😊  Just let me know what it is. 


